Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
!pip install numerapi
!pip install vecstack;

  Created wheel for vecstack: filename=vecstack-0.4.0-cp37-none-any.whl size=19877 sha256=56a49da13cb93786a221a9da7cbb771487aca264c50204caf6551fc575724c2a
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [3]:
##################################################################
##################### LIBRARIES ##################################
##################################################################

from google.colab import drive
drive.mount('drive')


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [4]:
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool

import numerapi

import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils 
from sklearn import preprocessing
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import math
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor 

import torch.nn as nn
import torch.nn.functional as F

from vecstack import stacking

from sklearn import metrics




In [5]:
'''
##################### CLASSES AND FUNCTIONS ##################################
'''
def my_loss_fn(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)
    
def Cluster_Kmeans(df,X,clusters):
    
    print('Call algorithm K-means')
    kmeans = KMeans(n_clusters=clusters, random_state=rand, init = 'random')
    print('Fitting algorithm K-means')
    kmeans.fit(X)
    print('Finished and Fitted')
    X_clusters_kmeans = df
    X_clusters_kmeans['k-means'] = kmeans.labels_
    return X_clusters_kmeans

def KMeans_Clustering_XGBRegressor(self,df,group,features):
  
    df_features_X = group[features]
    df_features_X['target'] = group.target
    X = df_features_X[df_features_X.columns[0:-1]]
    Y = df_features_X[df_features_X.columns[-1]]
    x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.20,random_state=rand)

    model = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, verbosity=1, nthread=6)
    model.fit(x_train, y_train)

    return model, x_train,x_test,y_train,y_test

def standardize_data(df):
    scaler = RobustScaler()
    data = scaler.fit_transform(df)
    return data
#https://joedorfman.github.io/2018/numerai/

def my_loss_fn_1(y_true, y_pred):
    loss = tf.keras.losses.poisson(y_true, y_pred)
    return loss


def my_loss_fn_2(y_true, y_pred):
    threshold = 1     
    error = y_true - y_pred     
    is_small_error = tf.abs(error) <= threshold     
    small_error_loss = tf.square(error) / 2     
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)

def my_loss_fn_3(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


def rank_correlation(y_true, y_pred):
    return np.nan_to_num(stats.spearmanr(y_true, y_pred, axis=1)[0], nan=-1)


def rmsle_1(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

def rae_1(y_true, y_pred):
    return 'RAE', np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(np.mean(y_true) - y_true)), False


def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5


#https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/sklearn_example.py
#https://github.com/chricke/Numerai-Keras/tree/master
#https://forum.numer.ai/t/example-for-using-tensorflow-keras-with-a-custom-callback/855


'''
y_pred=[…]
y_true=[…]
corr_coefs=[…] #310 correlation coefficients with y_pred

mse= mean(square(y_pred - y_true)
correlation_penalty= square(std(abs(corr_coefs)))
loss= mse + correlation_penalty
'''

'\ny_pred=[…]\ny_true=[…]\ncorr_coefs=[…] #310 correlation coefficients with y_pred\n\nmse= mean(square(y_pred - y_true)\ncorrelation_penalty= square(std(abs(corr_coefs)))\nloss= mse + correlation_penalty\n'

In [6]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [7]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [8]:
seed_1 = 1
rand_1 = np.random.seed(seed_1)
seed_2 = rand_1
rand_2 = np.random.seed(seed_2)
seed_3 = rand_2
rand_3 = np.random.seed(seed_3)

In [9]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

2021-07-03 21:28:51,498 INFO numerapi.utils: starting download
./numerai_dataset_271.zip: 100%|█████████▉| 411M/411M [00:22<00:00, 20.8MB/s]2021-07-03 21:29:13,982 INFO numerapi.base_api: unzipping file...
./numerai_dataset_271.zip: 411MB [00:39, 20.8MB/s]                           

In [10]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [11]:
%%time
print("# Loading data...")

training_data = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

# Loading data...
# All Loaded...
CPU times: user 49.9 s, sys: 7.47 s, total: 57.4 s
Wall time: 1min


In [12]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [13]:
X = training_data[feature_names]
Y = training_data[training_data.columns[-1]]

le = LabelEncoder()
Y_enc = le.fit_transform(Y)

In [14]:
%%time
print('Call algorithm K-means')
kmeans = KMeans(n_clusters=10, random_state=rand_1, init = 'random')
print('Fitting algorithm K-means')
kmeans.fit(X)
print('Finished and Fitted')
X_clusters_kmeans = training_data
X_clusters_kmeans['k-means'] = kmeans.labels_

Call algorithm K-means
Fitting algorithm K-means
Finished and Fitted
CPU times: user 10min 26s, sys: 3min 7s, total: 13min 34s
Wall time: 10min 5s


In [15]:
training_data["id_seq"] = list(range(len(training_data)))
tournament_data["id_seq"] = list(range(len(tournament_data)))

In [16]:
tournament_data

,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,...,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target,id_seq
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.00,0.75,0.50,0.25,0.50,0.50,0.25,0.00,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.00,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,0.25,...,0.00,0.25,0.50,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.50,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.50,1.00,0.00,0.00,0.25,0
n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.50,0.00,0.00,0.75,0.25,0.00,0.75,0.50,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.00,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,0.25,...,0.50,0.25,1.00,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.50,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.50,0.50,0.50,0.50,0.50,1
n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.50,0.50,1.00,1.00,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.50,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,1.00,...,0.50,0.00,0.00,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.00,0.50,0.50,0.00,1.00,2
n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.00,0.00,0.00,0.50,0.75,1.00,0.75,0.00,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.00,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,0.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25,0.50,3
n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.00,0.50,0.50,0.25,0.25,0.50,0.50,1.00,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.00,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,0.25,...,0.50,0.50,0.00,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.00,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.00,0.00,0.25,1.00,0.75,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffb091b15452291,eraX,live,0

In [17]:
X_group_1 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 0]
X_group_2 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 1]
X_group_3 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 2]
X_group_4 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 3]
X_group_5 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 4]
X_group_6 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 5]
X_group_7 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 6]
X_group_8 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 7]
X_group_9 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 8]
X_group_10 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 9]

2021-07-03 21:40:50,195 INFO numexpr.utils: NumExpr defaulting to 4 threads.


In [18]:
def Grouping_Models(df_train):
  
    df_train_all = df_train[feature_names]
    df_train_all[TARGET_NAME] = df_train['target'].values
    X = df_train_all[df_train_all.columns[0:-1]]
    Y = df_train_all[df_train_all.columns[-1]]
    x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.20,random_state=rand_3)
    model = LGBMRegressor(random_state=rand_1)
    model.fit(x_train, y_train, eval_set=[(x_test, y_test)])
    
    return model

In [19]:
model_group_1 = Grouping_Models(X_group_1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.0486476
[2]	valid_0's l2: 0.0486571
[3]	valid_0's l2: 0.0486522
[4]	valid_0's l2: 0.0486623
[5]	valid_0's l2: 0.0486544
[6]	valid_0's l2: 0.0486561
[7]	valid_0's l2: 0.0486354
[8]	valid_0's l2: 0.0486263
[9]	valid_0's l2: 0.0486222
[10]	valid_0's l2: 0.048634
[11]	valid_0's l2: 0.0486463
[12]	valid_0's l2: 0.0486581
[13]	valid_0's l2: 0.0486626
[14]	valid_0's l2: 0.0486655
[15]	valid_0's l2: 0.0486918
[16]	valid_0's l2: 0.0486918
[17]	valid_0's l2: 0.0486982
[18]	valid_0's l2: 0.0487148
[19]	valid_0's l2: 0.0487086
[20]	valid_0's l2: 0.0486918
[21]	valid_0's l2: 0.0487063
[22]	valid_0's l2: 0.0487207
[23]	valid_0's l2: 0.0487305
[24]	valid_0's l2: 0.0487298
[25]	valid_0's l2: 0.0487383
[26]	valid_0's l2: 0.0487455
[27]	valid_0's l2: 0.0487391
[28]	valid_0's l2: 0.0487538
[29]	valid_0's l2: 0.04877
[30]	valid_0's l2: 0.0487766
[31]	valid_0's l2: 0.0487823
[32]	valid_0's l2: 0.0487746
[33]	valid_0's l2: 0.0487768
[34]	valid_0's l2: 0.0487798
[35]	valid_0's l2: 0.04879

In [20]:
model_group_2 = Grouping_Models(X_group_2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.0458256
[2]	valid_0's l2: 0.0458231
[3]	valid_0's l2: 0.0458269
[4]	valid_0's l2: 0.0458176
[5]	valid_0's l2: 0.0458144
[6]	valid_0's l2: 0.0458019
[7]	valid_0's l2: 0.0457943
[8]	valid_0's l2: 0.0458018
[9]	valid_0's l2: 0.0458072
[10]	valid_0's l2: 0.0458103
[11]	valid_0's l2: 0.0458058
[12]	valid_0's l2: 0.0457973
[13]	valid_0's l2: 0.0457723
[14]	valid_0's l2: 0.0457773
[15]	valid_0's l2: 0.0457859
[16]	valid_0's l2: 0.0457791
[17]	valid_0's l2: 0.0457851
[18]	valid_0's l2: 0.045805
[19]	valid_0's l2: 0.0458147
[20]	valid_0's l2: 0.0458113
[21]	valid_0's l2: 0.0458119
[22]	valid_0's l2: 0.0457994
[23]	valid_0's l2: 0.0457968
[24]	valid_0's l2: 0.0457957
[25]	valid_0's l2: 0.0458137
[26]	valid_0's l2: 0.0458144
[27]	valid_0's l2: 0.0458143
[28]	valid_0's l2: 0.0458221
[29]	valid_0's l2: 0.0458298
[30]	valid_0's l2: 0.0458309
[31]	valid_0's l2: 0.0458461
[32]	valid_0's l2: 0.0458261
[33]	valid_0's l2: 0.0458199
[34]	valid_0's l2: 0.0458257
[35]	valid_0's l2: 0.045

In [21]:
model_group_2 = Grouping_Models(X_group_2)
model_group_3 = Grouping_Models(X_group_3)
model_group_4 = Grouping_Models(X_group_4)
model_group_5 = Grouping_Models(X_group_5)
model_group_6 = Grouping_Models(X_group_6)
model_group_7 = Grouping_Models(X_group_7)
model_group_8 = Grouping_Models(X_group_8)
model_group_9 = Grouping_Models(X_group_9)
model_group_10 = Grouping_Models(X_group_10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.0458727
[2]	valid_0's l2: 0.0458936
[3]	valid_0's l2: 0.0458951
[4]	valid_0's l2: 0.0458903
[5]	valid_0's l2: 0.0458898
[6]	valid_0's l2: 0.0458855
[7]	valid_0's l2: 0.0458875
[8]	valid_0's l2: 0.0458892
[9]	valid_0's l2: 0.045886
[10]	valid_0's l2: 0.0459066
[11]	valid_0's l2: 0.0459207
[12]	valid_0's l2: 0.045932
[13]	valid_0's l2: 0.0459447
[14]	valid_0's l2: 0.0459654
[15]	valid_0's l2: 0.0459814
[16]	valid_0's l2: 0.0459976
[17]	valid_0's l2: 0.0460197
[18]	valid_0's l2: 0.0460236
[19]	valid_0's l2: 0.0460326
[20]	valid_0's l2: 0.046029
[21]	valid_0's l2: 0.0460345
[22]	valid_0's l2: 0.0460428
[23]	valid_0's l2: 0.0460436
[24]	valid_0's l2: 0.0460451
[25]	valid_0's l2: 0.0460469
[26]	valid_0's l2: 0.0460591
[27]	valid_0's l2: 0.0460638
[28]	valid_0's l2: 0.0460661
[29]	valid_0's l2: 0.0460658
[30]	valid_0's l2: 0.0460678
[31]	valid_0's l2: 0.0460714
[32]	valid_0's l2: 0.0460798
[33]	valid_0's l2: 0.0460685
[34]	valid_0's l2: 0.0460703
[35]	valid_0's l2: 0.04606

In [22]:
########################## PREDICTIONS ON LEBELS ##############################

In [23]:
labels_tournament = kmeans.predict(tournament_data[feature_names])
tournament_data['k-means'] = labels_tournament

In [24]:
X_group_1_td = tournament_data.loc[tournament_data['k-means'] == 0]
X_group_2_td = tournament_data.loc[tournament_data['k-means'] == 1]
X_group_3_td = tournament_data.loc[tournament_data['k-means'] == 2]
X_group_4_td = tournament_data.loc[tournament_data['k-means'] == 3]
X_group_5_td = tournament_data.loc[tournament_data['k-means'] == 4]
X_group_6_td = tournament_data.loc[tournament_data['k-means'] == 5]
X_group_7_td = tournament_data.loc[tournament_data['k-means'] == 6]
X_group_8_td = tournament_data.loc[tournament_data['k-means'] == 7]
X_group_9_td = tournament_data.loc[tournament_data['k-means'] == 8]
X_group_10_td = tournament_data.loc[tournament_data['k-means'] == 9]

In [25]:
prediction_group_1 = pd.DataFrame()
prediction_group_1[PREDICTION_NAME] = model_group_1.predict(X_group_1_td[feature_names])
prediction_group_1["id_seq"] = X_group_1_td["id_seq"].values

In [26]:
prediction_group_2 = pd.DataFrame()
prediction_group_2[PREDICTION_NAME] = model_group_2.predict(X_group_2_td[feature_names])
prediction_group_2["id_seq"] = X_group_2_td["id_seq"].values

In [27]:
prediction_group_3 = pd.DataFrame()
prediction_group_3[PREDICTION_NAME] = model_group_3.predict(X_group_3_td[feature_names])
prediction_group_3["id_seq"] = X_group_3_td["id_seq"].values

In [28]:
prediction_group_4 = pd.DataFrame()
prediction_group_4[PREDICTION_NAME] = model_group_4.predict(X_group_4_td[feature_names])
prediction_group_4["id_seq"] = X_group_4_td["id_seq"].values

In [29]:
prediction_group_5 = pd.DataFrame()
prediction_group_5[PREDICTION_NAME] = model_group_5.predict(X_group_5_td[feature_names])
prediction_group_5["id_seq"] = X_group_5_td["id_seq"].values

In [30]:
prediction_group_6 = pd.DataFrame()
prediction_group_6[PREDICTION_NAME] = model_group_6.predict(X_group_6_td[feature_names])
prediction_group_6["id_seq"] = X_group_6_td["id_seq"].values

In [31]:
prediction_group_7 = pd.DataFrame()
prediction_group_7[PREDICTION_NAME] = model_group_7.predict(X_group_7_td[feature_names])
prediction_group_7["id_seq"] = X_group_7_td["id_seq"].values

In [32]:
prediction_group_8 = pd.DataFrame()
prediction_group_8[PREDICTION_NAME] = model_group_8.predict(X_group_8_td[feature_names])
prediction_group_8["id_seq"] = X_group_8_td["id_seq"].values

In [33]:
prediction_group_9 = pd.DataFrame()
prediction_group_9[PREDICTION_NAME] = model_group_9.predict(X_group_9_td[feature_names])
prediction_group_9["id_seq"] = X_group_9_td["id_seq"].values

In [34]:
prediction_group_10 = pd.DataFrame()
prediction_group_10[PREDICTION_NAME] = model_group_10.predict(X_group_10_td[feature_names])
prediction_group_10["id_seq"] = X_group_10_td["id_seq"].values

In [35]:
########################## UNION OF PREDICTIONS ##############################

In [36]:
vertical_stack = pd.concat([prediction_group_1, prediction_group_2,prediction_group_3,prediction_group_4,prediction_group_5
                            ,prediction_group_6,prediction_group_7,prediction_group_8,prediction_group_9
                            ,prediction_group_10], axis=0)
vertical_stack = vertical_stack.sort_values("id_seq")

In [37]:
tournament_data[PREDICTION_NAME] = vertical_stack[PREDICTION_NAME].values

In [38]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.477257
n000920ed083903f    0.436844
n0038e640522c4a6    0.493116
n004ac94a87dc54b    0.496139
n0052fe97ea0c05f    0.506500
                      ...   
nffb091b15452291    0.498464
nffc0338378383a7    0.494540
nffcfc256cb1ec1a    0.508156
nffdb7467dc351b9    0.548385
nffec2680b381924    0.511018
Name: prediction, Length: 1757987, dtype: float64

In [39]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [40]:
tournament_data[PREDICTION_NAME].to_csv("FisaGol_5.csv")

In [41]:
#save your prediction file locally

In [42]:
!cp FisaGol_5.csv "drive/My Drive/PHOENIXSIGMA/"